## tf fiddle with imdb sentiment dataset 
###### [last update: aug29/2018]
<hr><hr><br>

#### useful links etc.

- [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) for dataset `tf.keras.datasets.imdb`
- [explanation](https://stackoverflow.com/questions/42821330/restore-original-text-from-keras-s-imdb-dataset) of weird quasi-BoW format for input features
  - see accepted answer for ex of restoring text; 
  - see second answer for original text. 
- examples of analyses using this dataset:
  - ... \[insert\]
  - ... \[insert\]

In [ ]:
## this is 80 chars, just for reference (no vertical rule in jupyterlab?! </3)
## ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ # ~ #

### 1. setup 
<hr>

#### 1.1 imports 

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import tensorflow as tf

In [3]:
import numpy as np

#### 1.2 load imdb sentiment data

In [4]:
# load imdb sentiment data 
imdb = tf.keras.datasets.imdb

# ys: each a binary label, xs: each a review coded as int array
(x_train, y_train), (x_test, y_test) = imdb.load_data()

In [11]:

# x_train[0]
# wdict = imdb.get_word_index()
# wdict['happy']

x_train[:2]

# [len(x) for x in x_train[:10]]


array([ list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 

###### define some utilities for handling text

In [ ]:
# dict associating int indices w words 
wdict = imdb.get_word_index()

In [ ]:
# funcs for translating between words and indices 

# get a word's numeric index 
def w_to_idx(w):
  return wdict[w]

# get an index's corresponding word 
def idx_to_w(widx):
  return [w for w, idx in wdict.items() if idx == widx][0]

# TODO: 
#   - figure out which is more efficient for idx_to_w():
#       current; or 
#       next((w for w, idx in wdict.items() if idx==widx),None); or 
#       e.g. [w for w, idx in wdict.items() if idx in widxs]
#   - return individual idx/word or lists of them?? 

### 2. show structure of data and demo utils
<hr>

#### 2.1 basic data properties

In [ ]:
# size of test and train sets 
print(f'train n: {len(y_train)}, test n: {len(y_test)}')

In [ ]:
# first few train labels, unique label vals, and train freqs 
print(y_train[:10], np.unique(y_train, return_counts=True), sep='\n')

In [ ]:
# first train input 
# (corresponding label for x == y_train[0] == 1)
x = x_train[0] 

print(x[:10],     # first ten elements 
      len(x),     # length is 218
      type(x),    # x is a list... 
      type(x[0]), # of ints
      sep='\n')

#### 2.2 example usage for utils `w_to_idx()`, and `idx_to_w()`

In [ ]:
# example word indices and words 
widxs = [2289, 70691, 10092]
words = ['woody', 'boosh', 'yikes']

# get a word from idx, or vice versa 
print('idx', w_to_idx('woody'), 'is word', idx_to_w(2289))

# get a list of words from list of idxs, or vice versa 
print('words for `widxs`: ', [idx_to_w(idx) for idx in widxs])
print('idxs  for `words`: ', [w_to_idx(w) for w in words])

In [ ]:
# translate init segment of a review from idxs to words 
# [note that x inputs already just bags of lowercase words]
print('first five word indices:', x[:5])
print('word translations:', [idx_to_w(idx) for idx in x[:5]])

### 3. train + eval a couple of `sklearn` classifiers
<hr>

In [ ]:
from sklearn.metrics import accuracy_score

#### re-encode preprocessed features to facilitate usual sklearn workflow 

In [ ]:
### TODO: CURRENTLY THIS IS WAYYYYYY TOO SLOW. SO EITHER
###    - FIND A MORE EFFICIENT WAY; OR
###    - USE LINK IN NOTES AT TOP OF NB TO FIND RAWER VERSION...


# re-encode a doc as a string of its words (a 'quasi-document') 
def idxlist_to_quasi_doc(x):
  return ' '.join([idx_to_w(idx) for idx in x])

# since test/train are same size, can create qd-lists simultaneously
quasi_docs_train = []
quasi_docs_test = []

for xidx in range(len(x_train)): 
  if not xidx % 10: print(f'on iter `{xidx}` of `{len(x_test)}`...')
  quasi_docs_train.append(idxlist_to_quasi_doc(x_train[xidx]))
  quasi_docs_test.append(idxlist_to_quasi_doc(x_test[xidx]))


#### 3.1 multinomial naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf_mnb = MultinomialNB()
clf_mnb.fit(X=x_train, y=y_train)
# preds_mnb = clf_mnb.predict(X=x_test)
# accuracy_score(y_true=y_test, y_pred=preds_mnb)

#### 3.2 logistic regression 

In [ ]:
from sklearn.linear_model import LogisticRegression



#### 3.3 support vector machine  

In [ ]:
from sklearn.svm import SVC



#### 3.4 random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

